In [146]:
import spacy
nlp = spacy.load('en')

In [147]:
import gensim
import textacy

In [148]:
import pandas as pd
import numpy as np

In [149]:
df=pd.read_csv("https://raw.githubusercontent.com/susanli2016/Machine-Learning-with-Python/master/research_paper.csv")

In [150]:
df.head()

,Title,Conference
0,Innovation in Database Management: Computer Sc...,VLDB
1,High performance prime field multiplication fo...,ISCAS
2,enchanted scissors: a scissor interface for su...,SIGGRAPH
3,Detection of channel degradation attack by Int...,INFOCOM
4,Pinning a Complex Network through the Betweenn...,ISCAS


In [151]:
df['Conference'].value_counts()

ISCAS       864
INFOCOM     515
VLDB        423
WWW         379
SIGGRAPH    326
Name: Conference, dtype: int64

In [152]:
X_temp=df['Title'].values
y=df['Conference'].values

In [153]:
def sent_tokenize(sent):
    temp=nlp(sent)
    return [str(token) for token in temp if not token.is_stop] 

In [154]:
temp_sentences=[textacy.preprocess_text(str(sent), lowercase=True, no_punct=True) for sent in X_temp] 

In [155]:
X=[sent_tokenize(sent) for sent in temp_sentences]

In [156]:
from sklearn.model_selection import train_test_split

In [157]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42,stratify=y)

In [158]:
loc="./storage/cc.en.300.bin"

In [159]:
from gensim.models.wrappers import FastText

model = FastText.load_fasttext_format(loc)

In [160]:
def get_embedding(word):
    try:
        embedding=model[word]
    except:
        embedding=np.zeros((300,))
    return embedding

In [161]:
X_train_embeddings=[np.mean(np.array(list(map(get_embedding,tok_sent))),axis=0) for tok_sent in X_train]
X_test_embeddings=[np.mean(np.array(list(map(get_embedding,tok_sent))),axis=0) for tok_sent in X_test]

In [162]:
from sklearn.svm import SVC

In [163]:
clf=SVC(kernel='linear',C=3)

In [164]:
clf.fit(X_train_embeddings,y_train)

SVC(C=3, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [165]:
predictions=clf.predict(X_test_embeddings)

In [166]:
from sklearn.metrics import accuracy_score

In [167]:
accuracy_score(y_test,predictions)

0.7941567065073041

In [168]:
from sklearn import metrics
print(metrics.classification_report(y_test, predictions, 
                                    target_names=df['Conference'].unique()))

             precision    recall  f1-score   support

       VLDB       0.80      0.77      0.79       155
      ISCAS       0.84      0.90      0.87       259
   SIGGRAPH       0.73      0.72      0.73        98
    INFOCOM       0.76      0.78      0.77       127
        WWW       0.77      0.66      0.71       114

avg / total       0.79      0.79      0.79       753



In [169]:
clf1=SVC(kernel='rbf',C=5,gamma=3)

In [170]:
clf1.fit(X_train_embeddings,y_train)

SVC(C=5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=3, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [171]:
predictions=clf1.predict(X_test_embeddings)

In [172]:
accuracy_score(y_test,predictions)

0.8007968127490039

In [173]:
from sklearn.model_selection import GridSearchCV

In [174]:
params={'C':[0.01,0.1,1,10],'gamma':[0.01,0.1,1,5,10]}
clf_test=SVC(kernel='rbf')

In [175]:
cv=GridSearchCV(estimator=clf_test,param_grid=params,scoring='accuracy',cv=4,verbose=True)

In [176]:
cv.fit(X_train_embeddings,y_train)

Fitting 4 folds for each of 20 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:  2.0min finished


GridSearchCV(cv=4, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.01, 0.1, 1, 10], 'gamma': [0.01, 0.1, 1, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=True)

In [177]:
pd.DataFrame(cv.cv_results_).sort_values('mean_test_score',ascending=False).head()

/usr/local/lib/python3.5/dist-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/deprecation.py:1

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_C,param_gamma,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,split3_test_score,split3_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
16,0.497277,0.142346,0.784493,0.850816,10,0.1,"{'C': 10, 'gamma': 0.1}",1,0.775000,0.848554,0.783105,0.847264,0.792237,0.851064,0.787671,0.856383,0.002997,0.000786,0.006371,0.003492
12,0.549201,0.151517,0.774230,0.851956,1,1,"{'C': 1, 'gamma': 1}",2,0.770455,0.848554,0.771689,0.848784,0.785388,0.851064,0.769406,0.859422,0.002955,0.000702,0.006488,0.004421
17,0.412830,0.129302,0.774230,0.962371,10,1,"{'C': 10, 'gamma': 1}",2,0.781818,0.959665,0.760274,0.967325,0.789954,0.963526,0.764840,0.958967,0.007631,0.002959,0.012118,0.003346
18,0.881369,0.167953,0.751425,0.999620,10,5,"{'C': 10, 'gamma': 5}",4,0.772727,0.999239,0.712329,1.000000,0.771689,0.999240,0.748858,1.000000,0.004237,0.001306,0.024489,0.000380
13,0.676620,0.166475,0.748005,0.943748,1,5,"{'C': 1, 'gamma': 5}",5,0.761364,0.945967,0.728311,0.942249,0.764840,0.946049,0.737443,0.940729,0.003349,0.000903,0.015500,0.002322


In [178]:
cv.best_params_

{'C': 10, 'gamma': 0.1}

In [179]:
model=cv.best_estimator_

In [180]:
predictions=model.predict(X_test_embeddings)

In [181]:
accuracy_score(y_test,predictions)

0.7901726427622842

In [182]:
from sklearn import metrics
print(metrics.classification_report(y_test, predictions, 
                                    target_names=df['Conference'].unique()))

             precision    recall  f1-score   support

       VLDB       0.80      0.77      0.79       155
      ISCAS       0.83      0.90      0.86       259
   SIGGRAPH       0.75      0.71      0.73        98
    INFOCOM       0.73      0.78      0.75       127
        WWW       0.77      0.66      0.71       114

avg / total       0.79      0.79      0.79       753



In [183]:
from sklearn.ensemble import RandomForestClassifier

In [184]:
dt_clf=RandomForestClassifier(n_estimators=200,min_samples_split=15,min_samples_leaf=10,max_depth=30,random_state=40,class_weight='balanced')

In [185]:
dt_clf.fit(X_train_embeddings,y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=30, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=10,
            min_samples_split=15, min_weight_fraction_leaf=0.0,
            n_estimators=200, n_jobs=1, oob_score=False, random_state=40,
            verbose=0, warm_start=False)

In [186]:
predictions=dt_clf.predict(X_test_embeddings)

In [187]:
accuracy_score(y_test,predictions)

0.7689243027888446

In [188]:
import xgboost as xgb

In [189]:
from sklearn import preprocessing

In [190]:
lc=preprocessing.LabelEncoder()
lc.fit(y)

LabelEncoder()

In [191]:
y_train_enc=lc.transform(y_train)

In [192]:
y_test_enc=lc.transform(y_test)

In [193]:
train=xgb.DMatrix(X_train_embeddings,label=y_train_enc)
test=xgb.DMatrix(X_test_embeddings,label=y_test_enc)

In [194]:
param = {}

param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['max_depth'] = 30
param['scale_pos_weight'] = 10
param['num_class']=len(set(y))
param['colsample_bytree']=0.9
param['subsample']=0.9
param['eval_metric']='merror'

watchlist = [(train, 'train'), (test, 'test')]
num_round = 200
bst = xgb.train(param, train, num_round, watchlist)

[22:52:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 148 extra nodes, 0 pruned nodes, max_depth=11
[22:52:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 166 extra nodes, 0 pruned nodes, max_depth=10
[22:52:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 0 pruned nodes, max_depth=11
[22:52:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 148 extra nodes, 0 pruned nodes, max_depth=11
[22:52:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 128 extra nodes, 0 pruned nodes, max_depth=15
[0]	train-merror:0.068415	test-merror:0.427623
[22:52:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 172 extra nodes, 0 pruned nodes, max_depth=13
[22:52:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 182 extra nodes, 0 pruned nodes, max_depth=10
[22:52:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots,

[22:52:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 82 extra nodes, 0 pruned nodes, max_depth=10
[22:52:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 66 extra nodes, 0 pruned nodes, max_depth=9
[12]	train-merror:0	test-merror:0.260292
[22:52:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 0 pruned nodes, max_depth=12
[22:52:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 86 extra nodes, 0 pruned nodes, max_depth=8
[22:52:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 64 extra nodes, 0 pruned nodes, max_depth=8
[22:52:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 70 extra nodes, 0 pruned nodes, max_depth=9
[22:52:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 0 pruned nodes, max_depth=11
[13]	train-merror:0	test-merror:0.256308
[22:52:30] /workspace/src/tree/updater_prune.cc:74: tr

[22:52:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=7
[22:52:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=5
[22:52:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 0 pruned nodes, max_depth=6
[22:52:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 0 pruned nodes, max_depth=7
[25]	train-merror:0	test-merror:0.237716
[22:52:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=6
[22:52:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 0 pruned nodes, max_depth=7
[22:52:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 0 pruned nodes, max_depth=5
[22:52:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 0 p

[22:52:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=5
[37]	train-merror:0	test-merror:0.244356
[22:52:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=5
[22:52:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=5
[22:52:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=4
[22:52:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=6
[22:52:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=4
[38]	train-merror:0	test-merror:0.245684
[22:52:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=5
[22:52:31] /workspace/src/tree/updater_prune.cc:74: tree 

[22:52:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=4
[22:52:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[22:52:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=4
[22:52:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=4
[50]	train-merror:0	test-merror:0.237716
[22:52:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=4
[22:52:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=4
[22:52:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=4
[22:52:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 p

...

[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[140]	train-merror:0	test-merror:0.232404
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned n

[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[153]	train-merror:0	test-merror:0.227092
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned n

[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[166]	train-merror:0	test-merror:0.231076
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned n

[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[179]	train-merror:0	test-merror:0.231076
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned n

[22:52:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[22:52:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[192]	train-merror:0	test-merror:0.229748
[22:52:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[22:52:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned n

In [195]:
pred = bst.predict(test)

In [196]:
predictions=lc.inverse_transform(list(map(int,pred)))

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [197]:
accuracy_score(predictions,y_test)

0.7689243027888446

In [198]:
from sklearn.decomposition import PCA

In [233]:
pca=PCA(n_components=50)

In [234]:
pca.fit(X_train_embeddings)

PCA(copy=True, iterated_power='auto', n_components=50, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [235]:
X_train_pca=pca.transform(X_train_embeddings)
X_test_pca=pca.transform(X_test_embeddings)

In [236]:
clf3=SVC(kernel='rbf',C=5,gamma=1,random_state=42)

In [237]:
clf3.fit(X_train_pca,y_train)

SVC(C=5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)

In [238]:
predictions=clf3.predict(X_test_pca)

In [239]:
accuracy_score(predictions,y_test)

0.7875166002656042